## Lab 4, Group 4
### Names: Hailey DeMark, Deborah Park, Karis Park
### Student IDs: 48869449, 48878679, 48563429

Link to DataSet: https://www.kaggle.com/datasets/muonneutrino/us-census-demographic-data/data

## Load, Split, and Balance (1.5 points total)

* [.5 points] (1) Load the data into memory and save it to a pandas data frame. Do not normalize or one-hot encode any of the features until asked to do so later in the rubric. (2) Remove any observations that having missing data. (3) Encode any string data as integers for now. (4) You have the option of keeping the "county" variable or removing it. Be sure to discuss why you decided to keep/remove this variable. 

* // answer goes here

In [16]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [17]:
# Load Data 
file = 'data.csv'
df = pd.read_csv(file)

# remove missing rows
df.dropna(inplace=True)

# convert string to integer
categorical_columns = df.select_dtypes(include='object').columns
label_encoders = {}

for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# remove county
df = df.drop(columns=['County'])

# check
print("Shape of cleaned data:", df.shape)
print("Remaining columns:", df.columns.tolist())

Shape of cleaned data: (72718, 36)
Remaining columns: ['TractId', 'State', 'TotalPop', 'Men', 'Women', 'Hispanic', 'White', 'Black', 'Native', 'Asian', 'Pacific', 'VotingAgeCitizen', 'Income', 'IncomeErr', 'IncomePerCap', 'IncomePerCapErr', 'Poverty', 'ChildPoverty', 'Professional', 'Service', 'Office', 'Construction', 'Production', 'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp', 'WorkAtHome', 'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork', 'SelfEmployed', 'FamilyWork', 'Unemployment']


Should balancing of the dataset be done for both the training and testing set? Explain.
* Balancing of the dataset should only be done for the training set because we don't want to touch the testing set as much as possible. In a real-life scenario, we don't have access to the testing set, or future data, when training our model on the past data. So, we want to mimic these conditions as much as possible, leaving the testing data untouched until we're done training our model on the training data. 

In [19]:
# split features and target
y = df['ChildPoverty'].values
X = df.drop(columns=['ChildPoverty']).values

# train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# split 'childpoverty' into 4 quantiles
quantiles = np.quantile(y_train, [0.25, 0.5, 0.75])

def quant_poverty(val):
    if val <= quantiles[0]:
        return 0
    elif val <= quantiles[1]:
        return 1
    elif val <= quantiles[2]:
        return 2
    else:
        return 3

# Apply the function to the numpy array
vectorized_quant = np.vectorize(quant_poverty)
y_train_classes = vectorized_quant(y_train)
y_test_classes = vectorized_quant(y_test)

# One-hot encode the targets
y_train_dummy = np.eye(4)[y_train_classes]
y_test_dummy = np.eye(4)[y_test_classes]

# Check class distribution
print("Quantile thresholds:", quantiles)
print("Class distribution in training set:")
unique, counts = np.unique(y_train_classes, return_counts=True)
for class_label, count in zip(unique, counts):
    print(f"Class {class_label}: {count} instances")

(58174, 35) (14544, 35) (58174,) (14544,)
Quantile thresholds: [ 6.2 16.4 31.7]
Class distribution in training set:
Class 0: 14596 instances
Class 1: 14580 instances
Class 2: 14476 instances
Class 3: 14522 instances


## Pre-processing and Initial Modeling (2.5 points total)

You will be using a two layer perceptron from class for the next few parts of the rubric. There are several versions of the two layer perceptron covered in class, with example code. When selecting an example two layer network from class be sure that you use: (1) vectorized gradient computation, (2) mini-batching, (3) cross entropy loss, and (4) proper Glorot initialization, at a minimum. There is no need to use momentum or learning rate reduction (assuming you choose a sufficiently small learning rate). It is recommended to use sigmoids throughout the network, but not required.

* [.5 points] Use the example two-layer perceptron network from the class example and quantify performance using accuracy. Do not normalize or one-hot encode the data (not yet). Be sure that training converges by graphing the loss function versus the number of epochs. 

* [.5 points] Now (1) normalize the continuous numeric feature data. Use the example two-layer perceptron network from the class example and quantify performance using accuracy. Be sure that training converges by graphing the loss function versus the number of epochs.  

* [.5 points] Now(1) normalize the continuous numeric feature data AND (2) one hot encode the categorical data. Use the example two-layer perceptron network from the class example and quantify performance using accuracy. Be sure that training converges by graphing the loss function versus the number of epochs. 

* [1 points] Compare the performance of the three models you just trained. Are there any meaningful differences in performance? Explain, in your own words, why these models have (or do not have) different performances.  
    * Use one-hot encoding and normalization on the dataset for the remainder of this lab assignment.
We chose a customer segmentation dataset with 8,068 entries. It includes details about customers, such as their gender, age, marital status, education, job, work experience, spending habits, and family size. The goal is to predict which of four customer groups (A, B, C, or D) a customer belongs to based on their information. This dataset would be helpful for businesses to understand their customers and target audience better. Additionally, companies can use the results to send personalized offers, improve customer service, and make better marketing plans. This model would be deployed mostly for online-use because it's important for companies to keep up with the trends and preferences of their target audience and general customers to make the right choices in marketing and keep up sales. 

## Modeling (5 points total)

* [1 points] Add support for a third layer in the multi-layer perceptron. Add support for saving (and plotting after training is completed) the average magnitude of the gradient for each layer, for each epoch (like we did in the flipped module for back propagation). For magnitude calculation, you are free to use either the average absolute values or the L1/L2 norm.
    * Quantify the performance of the model and graph the magnitudes for each layer versus the number of epochs.

* [1 points] Repeat the previous step, adding support for a fourth layer.

* [1 points] Repeat the previous step, adding support for a fifth layer. 

* [2 points] Implement an adaptive learning technique that was discussed in lecture and use it on the five layer network (choose either RMSProp or AdaDelta). Discuss which adaptive method you chose. Compare the performance of your five layer model with and without the adaptive learning strategy. Do not use AdaM for the adaptive learning technique as it is part of the exceptional work.

## Exceptional Work (1 points total)

5000 level student: You have free reign to provide additional analyses.
One idea (required for 7000 level students):  Implement adaptive momentum (AdaM) in the five layer neural network and quantify the performance compared to other methods.  